In [1]:
import os
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pickle  # library for importing our machine learning model
import pandas as pd
import numpy as np
from sklearn import preprocessing


# load the model from disk
loaded_model = pickle.load(open("finalized_model.sav", "rb"))


# load the data from disk
loaded_data = pickle.load(open("unnormalized_data.sav", "rb"))

In [ ]:
app = dash.Dash(__name__)

#server class is required for publishing the app via heroku
#server = app.server

app.title = "PHYSJA: Heart Disease Prediction"

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.P(children="❤️", className="header-emoji"),
                html.H1(
                    children="PHYSJA: Heart Disease Prediction Tool",
                    className="header-title",
                ),
                html.P(
                    children="Enter patient details below to the nearest whole number: Age in years and Cholesterol in mg/dl",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Gender", className="menu-title"),
                        dcc.Dropdown(
                            id="gender-input",
                            options=[
                                {"label": "Male", "value": 1},
                                {"label": "Female", "value": 0},
                            ],
                            value="",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(children="Age", className="menu-title-ph"),
                        dcc.Input(
                            id="age-input",
                            type="number",
                            min=10,
                            max=150,
                            step=1,
                            value="",
                            className="Select-control",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(children="Chest Pain", className="menu-title"),
                        dcc.Dropdown(
                            id="pain-input",
                            options=[
                                {"label": "Typical angina", "value": 1},
                                {"label": "Atypical angina", "value": 2},
                                {"label": "Non anginal pain", "value": 3},
                                {"label": "Asymptomatic", "value": 4},
                            ],
                            value="",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(children="ECG", className="menu-title"),
                        dcc.Dropdown(
                            id="ecg-input",
                            options=[
                                {"label": "Normal", "value": 0},
                                {"label": "ST wave abnormality", "value": 1},
                                {
                                    "label": "Probable or definite left ventricular hypertrophy",
                                    "value": 2,
                                },
                            ],
                            value="",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(children="Cholesterol", className="menu-title-ph"),
                        dcc.Input(
                            id="cholesterol-input",
                            type="number",
                            step=1,
                            value="",
                            className="Select-control",
                        ),
                    ],
                ),
            ],
            className="menu",
        ),
        html.Div(
            [
                html.Button(
                    "Submit", id="submit-button", n_clicks=0, className="submit-button"
                ),
            ]
        ),
        html.Div(
            children=[
                html.H2(
                    id="output-prediction",
                    children="Enter Data and Press Submit Button",
                    className="output",
                ),
            ],
            className="wrapper",
        ),
    ]
)

# callback
@app.callback(
    Output(component_id="output-prediction", component_property="children"),
    Input(component_id="submit-button", component_property="n_clicks"),
    State(component_id="gender-input", component_property="value"),
    State(component_id="age-input", component_property="value"),
    State(component_id="pain-input", component_property="value"),
    State(component_id="ecg-input", component_property="value"),
    State(component_id="cholesterol-input", component_property="value"),
)
def update_output_div(n_clicks, gender, age, pain, ecg, cholesterol):
    new_input = [age, gender, pain, cholesterol, ecg]
    loaded_data.drop(columns=["Class"], inplace=True, errors="ignore")
    loaded_values = loaded_data.values
    new_values = np.vstack([loaded_values, new_input])
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(new_values)
    new_data = pd.DataFrame(x_scaled, columns=loaded_data.columns)
    new_input_normalized = new_data.tail(1).to_numpy()
    new_output = loaded_model.predict(new_input_normalized)
    if new_output == 1:
        return "Please do further investigations"
    elif new_output == 0:
        return "No risk identified from the system"


if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Oct/2021 15:36:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 15:36:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 15:36:47] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/janlorenz/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/janlorenz/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/janlorenz/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/janlorenz/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/janlorenz/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/janlorenz/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Us

127.0.0.1 - - [22/Oct/2021 15:36:47] "POST /_dash-update-component HTTP/1.1" 500 -
